<a href="https://colab.research.google.com/github/kkokay07/genomicclass/blob/master/NGS_1_Variant_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Environment
"""First, we need to install the required bioinformatics tools"""

In [1]:
# install required tools
!apt-get update && apt-get install -y fastp bwa samtools bcftools

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,514 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,454 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2

In [ ]:
# install SRA toolkit to download samples
!wget https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/current/sratoolkit.current-ubuntu64.tar.gz
!tar -xzf sratoolkit.current-ubuntu64.tar.gz
!mv sratoolkit.*/bin/* /usr/local/bin/

# 1. Understanding Input Data: FASTQ Format

FASTQ files contain sequencing reads with quality scores. Each read has 4 lines:
1. Read identifier (starts with @)
2. Sequence
3. '+' separator
4. Quality scores (ASCII-encoded)

Example of FASTQ format:
@SRR001666.1
GATTTGGGGTTCAAAGCAGTATCGATCAAATAGTAAATCCATTTGTTCAACTCACAGTTT
+
!''*((((***+))%%%++)(%%%%).1***-+*''))**55CCF>>>>>>CCCCCCC65

Let's download a small test dataset (E. coli K-12): This tutorial demonstrates variant calling with two samples, each with paired-end reads.
Sample1 and Sample2
Each sample has forward (R1) and reverse (R2) reads.

In [8]:
# Download first sample
!fastq-dump --split-files --gzip SRR14600276
!mv SRR14600276_1.fastq.gz original_R1.fastq.gz
!mv SRR14600276_2.fastq.gz original_R2.fastq.gz

# Verify the files exist and check sizes
!ls -lh original_*.fastq.gz

# Now split original data into different datasets/samples (for demonstration only; bcoz working with original files needs more computaional space)
!zcat original_R1.fastq.gz | head -n 4000 | gzip > sample1_R1.fastq.gz
!zcat original_R1.fastq.gz | tail -n 4000 | gzip > sample2_R1.fastq.gz
!zcat original_R2.fastq.gz | head -n 4000 | gzip > sample1_R2.fastq.gz
!zcat original_R2.fastq.gz | tail -n 4000 | gzip > sample2_R2.fastq.gz

# Verify all final files
!ls -lh *.fastq.gz

Read 783371 spots for SRR14600276
Written 783371 spots for SRR14600276
-rw-r--r-- 1 root root 20M Nov 30 02:46 original_R1.fastq.gz
-rw-r--r-- 1 root root 20M Nov 30 02:46 original_R2.fastq.gz
-rw-r--r-- 1 root root 20M Nov 30 02:46 original_R1.fastq.gz
-rw-r--r-- 1 root root 20M Nov 30 02:46 original_R2.fastq.gz
-rw-r--r-- 1 root root 26K Nov 30 02:46 sample1_R1.fastq.gz
-rw-r--r-- 1 root root 26K Nov 30 02:46 sample1_R2.fastq.gz
-rw-r--r-- 1 root root 27K Nov 30 02:46 sample2_R1.fastq.gz
-rw-r--r-- 1 root root 28K Nov 30 02:46 sample2_R2.fastq.gz


# 2. Reference Genome

We need a reference genome to align our reads. For E. coli K-12, we'll download it from NCBI:

In [10]:
# Download and prepare reference genome
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz
!gunzip GCF_000005845.2_ASM584v2_genomic.fna.gz
!mv GCF_000005845.2_ASM584v2_genomic.fna reference.fasta

# Verify the reference genome file exists
!ls -lh reference.fasta

--2024-11-30 02:47:53--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.11, 130.14.250.12, 130.14.250.13, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1379902 (1.3M) [application/x-gzip]
Saving to: ‘GCF_000005845.2_ASM584v2_genomic.fna.gz’

GCF_000005845.2_ASM 100%[===================>]   1.32M  1003KB/s    in 1.3s    

2024-11-30 02:47:56 (1003 KB/s) - ‘GCF_000005845.2_ASM584v2_genomic.fna.gz’ saved [1379902/1379902]

-rw-r--r-- 1 root root 4.5M Oct 31  2014 reference.fasta


# 3. Quality Control with fastp

fastp performs quality control and filtering on FASTQ files.
- Removes low-quality bases
- Trims adapters
- Generates QC reports

Input:
- Raw FASTQ files


Output:
- Cleaned FASTQ files
- HTML report with quality metrics

In [11]:
# Process Sample 1
!fastp -i sample1_R1.fastq.gz \
       -I sample1_R2.fastq.gz \
       -o cleaned_sample1_R1.fastq.gz \
       -O cleaned_sample1_R2.fastq.gz \
       --html sample1_fastp_report.html

Read1 before filtering:
total reads: 1000
total bases: 38000
Q20 bases: 35036(92.2%)
Q30 bases: 34326(90.3316%)

Read2 before filtering:
total reads: 1000
total bases: 38000
Q20 bases: 35151(92.5026%)
Q30 bases: 34448(90.6526%)

Read1 after filtering:
total reads: 998
total bases: 37924
Q20 bases: 34982(92.2424%)
Q30 bases: 34272(90.3702%)

Read2 aftering filtering:
total reads: 998
total bases: 37924
Q20 bases: 35101(92.5562%)
Q30 bases: 34398(90.7025%)

Filtering result:
reads passed filter: 1996
reads failed due to low quality: 2
reads failed due to too many N: 2
reads failed due to too short: 0
reads with adapter trimmed: 0
bases trimmed due to adapters: 0

Duplication rate: 0%

Insert size peak (evaluated by paired-end reads): 0

JSON report: fastp.json
HTML report: sample1_fastp_report.html

fastp -i sample1_R1.fastq.gz -I sample1_R2.fastq.gz -o cleaned_sample1_R1.fastq.gz -O cleaned_sample1_R2.fastq.gz --html sample1_fastp_report.html 
fastp v0.20.1, time used: 1 seconds


In [12]:
# Process Sample 2
!fastp -i sample2_R1.fastq.gz \
       -I sample2_R2.fastq.gz \
       -o cleaned_sample2_R1.fastq.gz \
       -O cleaned_sample2_R2.fastq.gz \
       --html sample2_fastp_report.html

Read1 before filtering:
total reads: 1000
total bases: 38000
Q20 bases: 34256(90.1474%)
Q30 bases: 32661(85.95%)

Read2 before filtering:
total reads: 1000
total bases: 38000
Q20 bases: 34136(89.8316%)
Q30 bases: 31925(84.0132%)

Read1 after filtering:
total reads: 994
total bases: 37772
Q20 bases: 34100(90.2785%)
Q30 bases: 32523(86.1035%)

Read2 aftering filtering:
total reads: 994
total bases: 37772
Q20 bases: 34004(90.0244%)
Q30 bases: 31817(84.2344%)

Filtering result:
reads passed filter: 1988
reads failed due to low quality: 12
reads failed due to too many N: 0
reads failed due to too short: 0
reads with adapter trimmed: 0
bases trimmed due to adapters: 0

Duplication rate: 0%

Insert size peak (evaluated by paired-end reads): 0

JSON report: fastp.json
HTML report: sample2_fastp_report.html

fastp -i sample2_R1.fastq.gz -I sample2_R2.fastq.gz -o cleaned_sample2_R1.fastq.gz -O cleaned_sample2_R2.fastq.gz --html sample2_fastp_report.html 
fastp v0.20.1, time used: 1 seconds


# 4. Preparing Reference Genome

Before alignment, we need to index the reference genome:

In [13]:
!bwa index reference.fasta

[bwa_index] Pack FASTA... 0.04 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 1.94 seconds elapse.
[bwa_index] Update BWT... 0.03 sec
[bwa_index] Pack forward-only FASTA... 0.02 sec
[bwa_index] Construct SA from BWT and Occ... 0.89 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index reference.fasta
[main] Real time: 3.000 sec; CPU: 2.921 sec


# 5. Alignment with BWA-MEM

BWA-MEM aligns cleaned reads to the reference genome.

Input: Cleaned FASTQ files + indexed reference

Output: SAM file (Sequence Alignment/Map format)

Example SAM format:
@SQ     SN:ref  LN:45
r001    163     ref     7       30      8M2I4M1D3M      =       37      39      TTAGATAAAGGATACTG     *


In [14]:
# Align Sample 1
!bwa mem reference.fasta cleaned_sample1_R1.fastq.gz cleaned_sample1_R2.fastq.gz > aligned_sample1.sam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 1996 sequences (75848 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 0, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] skip orientation FR as there are not enough pairs
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 1996 reads in 0.061 CPU sec, 0.065 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem reference.fasta cleaned_sample1_R1.fastq.gz cleaned_sample1_R2.fastq.gz
[main] Real time: 0.087 sec; CPU: 0.078 sec


In [15]:
# Align Sample 2
!bwa mem reference.fasta cleaned_sample2_R1.fastq.gz cleaned_sample2_R2.fastq.gz > aligned_sample2.sam


[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 1988 sequences (75544 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 0, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] skip orientation FR as there are not enough pairs
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 1988 reads in 0.042 CPU sec, 0.042 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem reference.fasta cleaned_sample2_R1.fastq.gz cleaned_sample2_R2.fastq.gz
[main] Real time: 0.060 sec; CPU: 0.055 sec


# 6. SAM to BAM Conversion

Convert SAM to BAM (binary format), sort, and index for efficient processing:

In [16]:
# Process Sample 1
!samtools view -bS aligned_sample1.sam > aligned_sample1.bam
!samtools sort aligned_sample1.bam -o sorted_sample1.bam
!samtools index sorted_sample1.bam


In [17]:
# Process Sample 2
!samtools view -bS aligned_sample2.sam > aligned_sample2.bam
!samtools sort aligned_sample2.bam -o sorted_sample2.bam
!samtools index sorted_sample2.bam

# 7. Variant Calling with BCFtools

Call variants from the aligned reads.

Input: Sorted BAM files + reference

Output: VCF file (Variant Call Format)

Example VCF format:
CHROM  POS     ID      REF     ALT     QUAL    FILTER  INFO
20      14370   rs6054257       G       A       29      PASS    NS=3;DP=14;AF=0.5

In [20]:
# Joint variant calling
# First, let's create the bam_list.txt file
!ls sorted_sample*.bam > bam_list.txt

# Verify the content of bam_list.txt
!cat bam_list.txt

# Now run joint variant calling
!bcftools mpileup -Ou -f reference.fasta -b bam_list.txt | bcftools call -mv -Ov -o variants_joint.vcf

# Alternative approach without using bam_list.txt:
#!bcftools mpileup -Ou -f reference.fasta sorted_sample1.bam sorted_sample2.bam | bcftools call -mv -Ov -o variants_joint.vcf

# Verify the output VCF file was created
#!ls -lh variants_joint.vcf


sorted_sample1.bam
sorted_sample2.bam
[mpileup] 2 samples in 2 input files
Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid
[mpileup] maximum number of reads per input file set to -d 250


# 8. Examining output files to understand output file format

1. fastp_report.html: Quality control metrics and graphs
2. cleaned_R1/R2.fastq.gz: Filtered reads
3. sorted.bam: Aligned reads in binary format
4. variants.vcf: Final variants in VCF format

Key VCF Fields:
- CHROM: Chromosome
- POS: Position in reference
- REF: Reference base(s)
- ALT: Alternative base(s)
- QUAL: Quality score
- FILTER: Whether variant passed filters


In [21]:
# View VCF header and first few variants
!head -n 20 variants_joint.vcf

##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##bcftoolsVersion=1.13+htslib-1.13+ds
##bcftoolsCommand=mpileup -Ou -f reference.fasta -b bam_list.txt
##reference=file://reference.fasta
##contig=<ID=NC_000913.3,length=4641652>
##ALT=<ID=*,Description="Represents allele(s) other than observed.">
##INFO=<ID=INDEL,Number=0,Type=Flag,Description="Indicates that the variant is an INDEL.">
##INFO=<ID=IDV,Number=1,Type=Integer,Description="Maximum number of raw reads supporting an indel">
##INFO=<ID=IMF,Number=1,Type=Float,Description="Maximum fraction of raw reads supporting an indel">
##INFO=<ID=DP,Number=1,Type=Integer,Description="Raw read depth">
##INFO=<ID=VDB,Number=1,Type=Float,Description="Variant Distance Bias for filtering splice-site artefacts in RNA-seq data (bigger is better)",Version="3">
##INFO=<ID=RPBZ,Number=1,Type=Float,Description="Mann-Whitney U-z test of Read Position Bias (closer to 0 is better)">
##INFO=<ID=MQBZ,Number=1,Type=Float,Description=

In [22]:
# Get basic statistics
!bcftools stats variants_joint.vcf | head -n 30

# This file was produced by bcftools stats (1.13+htslib-1.13+ds) and can be plotted using plot-vcfstats.
# The command line was:	bcftools stats  variants_joint.vcf
#
# Definition of sets:
# ID	[2]id	[3]tab-separated file names
ID	0	variants_joint.vcf
# SN, Summary numbers:
#   number of records   .. number of data rows in the VCF
#   number of no-ALTs   .. reference-only sites, ALT is either "." or identical to REF
#   number of SNPs      .. number of rows with a SNP
#   number of MNPs      .. number of rows with a MNP, such as CC>TT
#   number of indels    .. number of rows with an indel
#   number of others    .. number of rows with other type, for example a symbolic allele or
#                          a complex substitution, such as ACT>TCGA
#   number of multiallelic sites     .. number of rows with multiple alternate alleles
#   number of multiallelic SNP sites .. number of rows with multiple alternate alleles, all SNPs
# 
#   Note that rows containing multiple types will be coun

# 9. Sample-Specific Analysis
Let's look at variants in each sample:

In [23]:
# Count variants in sample 1
!echo "Variants in Sample 1:"
!bcftools view -s sample1 variants_joint.vcf | grep -v "#" | wc -l

Variants in Sample 1:
Error: subset called for sample that does not exist in header: "sample1". Use "--force-samples" to ignore this error.
0


In [ ]:
# Count variants in sample 2
!echo "Variants in Sample 2:"
!bcftools view -s sample2 variants_joint.vcf | grep -v "#" | wc -l

# 10. Compare Samples
Find variants unique to each sample:

In [ ]:
# Variants unique to Sample 1
!bcftools view -s sample1 variants_joint.vcf | bcftools view -e 'GT="0/0" || GT="./."' > sample1_unique.vcf


In [ ]:
# Variants unique to Sample 2
!bcftools view -s sample2 variants_joint.vcf | bcftools view -e 'GT="0/0" || GT="./."' > sample2_unique.vcf


# Take home message
File Organization:
Input Files:
- sample1_R1.fastq.gz, sample1_R2.fastq.gz (Sample 1 paired-end reads)
- sample2_R1.fastq.gz, sample2_R2.fastq.gz (Sample 2 paired-end reads)
- reference.fasta (Reference genome)

Intermediate Files:
- cleaned_sample{1,2}_R{1,2}.fastq.gz (Cleaned reads)
- sample{1,2}_fastp_report.html (QC reports)
- aligned_sample{1,2}.sam (Alignment files)
- sorted_sample{1,2}.bam (Sorted alignments)

Output Files:
- variants_joint.vcf (Combined variants)
- sample{1,2}_unique.vcf (Sample-specific variants)